In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
from ExtractGenre import *
from CNN_ExtractGenre import *
import Util as Util

import DatasetLoader as DL

In [5]:
InputPath = os.path.realpath('YAMF/genres_original')

GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [ ]:
# with open('GenreDataset.pkl', 'rb') as f:
#    GenreDataset = pickle.load(f)

In [ ]:
#CNN_GenreDataset = DiscriminateSongGenre(GenreDataset)

100%|██████████| 13164/13164 [02:01<00:00, 108.26it/s]


2317


In [ ]:
# with open('CNN_GenreDataset.pkl', 'rb') as f:
#    CNN_GenreDataset = pickle.load(f)

In [ ]:
# with open('CNN_GenreDataset.pkl', 'wb') as f:
#    pickle.dump(CNN_GenreDataset, f)

In [ ]:
#Dataset = torch.load('Dataset.pt', map_location='cpu')


In [ ]:
def ToBars(track, TicksPerBeat, length = 16):

   #since these tracks are all 4/4
   TicksPerBar = TicksPerBeat * 4
   TicksPerSixteenth = TicksPerBar // length

   currTime = 0
   Note = []

   for msg in track:
      currTime += msg.time
      if msg.type == 'note_on' and msg.velocity > 0:
         barNumber = currTime // TicksPerBar
         posInBar = (currTime % TicksPerBar) // TicksPerSixteenth
   
         if posInBar < length:
            Note.append((barNumber, msg.note, posInBar))

   Bars = {}
   for barNum, note, pos in Note:
      if barNum not in Bars:
         Bars[barNum] = np.zeros((128, length), dtype = int)

      #Fill the matrix with the note at it's correct position
      Bars[barNum][note, pos] = 1

   barList = []

   for barNum, matrix in Bars.items():
      if len(np.where(np.ravel(matrix) != 0)[0]) >= 5:
         Tensor = torch.tensor(matrix, dtype=torch.int)
         barList.append(Tensor.to_sparse())

   #Keeping only 10 sequential random bars
   # if len(barList) > 10:
   #    maxLen = len(barList)
   #    rIdx = np.random.randint(0, maxLen-10)
   #    FinalBarList = barList[rIdx:rIdx+10]
   
   return barList

In [ ]:
def ToGeneralInfo(mid, Dataset, file):

   Func_Tempo = lambda t: 60_000_000 / t
   TicksPerBeat = mid.ticks_per_beat

   #defining the tempo of file (one for each)
   if len(mid.tracks) > 0:
      InitialTrack = mid.tracks[0]
      for msg in InitialTrack:
         if msg.type == 'set_tempo':

            Tempo = Func_Tempo(msg.tempo)
            break
         else:
            Tempo = 120

   #Loop over all tracks (beside the first --> metadata)
   for track in mid.tracks[1:]:
      #Consider only the tracks that have an instrument in it (remove grabage)
      HasProgramChange = any(msg.type == 'program_change' for msg in track)
      TrackName = f'{file[:-4]}'

      if HasProgramChange:

         Program = [msg.program for msg in track if msg.type == 'program_change'][0]
         Channel = [msg.channel for msg in track if msg.type == 'program_change'][0]

         if Program == 0 or Channel == 10:
            continue

         #Compute the (128x16) bars matrix for each track
         Bars = ToBars(track, TicksPerBeat)

         if Bars is None or len(Bars) < 2:
            continue

         #Counts the number of pair of bars
         numPair = [(i, i+1) for i in range(0, len(Bars)//2 - 1, 2)]
         BarsPair = [(Bars[i], Bars[i+1]) for i in range(0, len(Bars)//2 - 1, 2)]


         #If there is not the track in the dataset, add it
         if TrackName not in Dataset:               
            Dataset[TrackName] = {
               'Bars': [],
               'Tempo': [], 
               'Program': [], 
               'Channel': [], 
               'SongName': [],
               'numBar': [] 
            }

         #Maps the program into one instrument of the same category
         NewProgram = Util.InstrumentMap[Program]
         
         #and add the information to the Dataset dictionary
         Dataset[TrackName]['Bars'].extend(BarsPair)
         Dataset[TrackName]['Tempo'].extend([(int(Tempo), int(Tempo)) for _ in range(0, len(Bars)//2-1, 2)])
         Dataset[TrackName]['Program'].extend([(NewProgram, NewProgram) for _ in range(0, len(Bars)//2-1, 2)])
         Dataset[TrackName]['Channel'].extend([(Channel, Channel) for _ in range(0, len(Bars)//2-1, 2)])
         Dataset[TrackName]['SongName'].extend([(f'{TrackName}', f'{TrackName}') for _ in range(0, len(Bars)//2-1, 2)])
         Dataset[TrackName]['numBar'].extend(numPair)

   return Dataset

In [ ]:
def PreProcessing(nDir = 300):

   Dataset = {}

   InputPath = os.path.relpath('Mono_CleanMidi')

   #Selecting a random number of directory
   all_dirs = [d for d in os.listdir(InputPath) if os.path.isdir(os.path.join(InputPath, d))]

   random_dirs = np.random.choice(all_dirs, nDir)

   for dir in tqdm(random_dirs, desc='Preprocessing'):
      DirPath = os.path.join(InputPath, dir)

      if not os.path.isdir(DirPath):
         continue

      #Real all the file in each folder
      for file in os.listdir(DirPath):

         FilePath = os.path.join(DirPath, file)

         #Cleaned monophonic: Some songs are corrupted:
         mid = Func_CorruptedFile(FilePath, file, dir)

         if mid is None:
            continue

         Dataset = ToGeneralInfo(mid, Dataset, file)